In [7]:
import os
import io
import shutil
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    PreTrainedTokenizer,
    VisionEncoderDecoderModel,
    ViTImageProcessor
)
from datasets import load_dataset  # หรือ load_from_disk
import sentencepiece as spm
from PIL import Image

print("✅ Libraries imported")

✅ Libraries imported


In [8]:
class ThaiTokenizerFixed(PreTrainedTokenizer):
    """Fixed Thai SentencePiece Tokenizer with proper special token handling"""
    
    vocab_files_names = {"vocab_file": "spm.model"}
    
    def __init__(self, vocab_file=None, **kwargs):
        self.vocab_file = vocab_file or 'thai_sp_fixed.model'
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(self.vocab_file)
        
        super().__init__(
            pad_token="<pad>",
            unk_token="<unk>",
            bos_token="<s>",
            eos_token="</s>",
            **kwargs
        )
    
    @property
    def pad_token_id(self):
        return 0
    
    @property
    def unk_token_id(self):
        return 1
    
    @property
    def bos_token_id(self):
        return 2
    
    @property
    def eos_token_id(self):
        return 3
    
    @property
    def vocab_size(self):
        return self.sp.vocab_size()
    
    def get_vocab(self):
        return {self.sp.id_to_piece(i): i for i in range(self.sp.vocab_size())}
    
    def _tokenize(self, text):
        return self.sp.encode_as_pieces(text)
    
    def _convert_token_to_id(self, token):
        return self.sp.piece_to_id(token)
    
    def _convert_id_to_token(self, index):
        return self.sp.id_to_piece(index)
    
    def convert_tokens_to_string(self, tokens):
        return self.sp.decode_pieces(tokens)
    
    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        bos = [self.bos_token_id]
        eos = [self.eos_token_id]
        if token_ids_1 is None:
            return bos + token_ids_0 + eos
        return bos + token_ids_0 + eos + bos + token_ids_1 + eos
    
    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        if already_has_special_tokens:
            return super().get_special_tokens_mask(
                token_ids_0=token_ids_0, 
                token_ids_1=token_ids_1, 
                already_has_special_tokens=True
            )
        if token_ids_1 is None:
            return [1] + ([0] * len(token_ids_0)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1] + [1] + ([0] * len(token_ids_1)) + [1]
    
    def save_vocabulary(self, save_directory, filename_prefix=None):
        if not os.path.isdir(save_directory):
            os.makedirs(save_directory)
        out_file = os.path.join(
            save_directory,
            (filename_prefix + "-" if filename_prefix else "") + "spm.model"
        )
        if os.path.abspath(self.vocab_file) != os.path.abspath(out_file):
            shutil.copy(self.vocab_file, out_file)
        return (out_file,)

print("✅ ThaiTokenizerFixed class defined")

✅ ThaiTokenizerFixed class defined


In [9]:
class ThaiHandwritingDatasetFixed(Dataset):
    """Dataset with proper BOS/EOS tokens"""
    def __init__(self, dataset, tokenizer, image_processor, max_length=128):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.image_processor = image_processor
        self.max_length = max_length
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        item = self.dataset[idx]
        
        # Process image
        image = item['image']
        if isinstance(image, dict):
            image = Image.open(io.BytesIO(image['bytes']))
        elif not isinstance(image, Image.Image):
            image = Image.fromarray(image)
        
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        pixel_values = self.image_processor(image, return_tensors="pt").pixel_values.squeeze(0)
        
        # Process text with special tokens
        text = item['text']
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            add_special_tokens=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].squeeze(0)
        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return {
            'pixel_values': pixel_values,
            'labels': labels,
            'text': text,
        }

def collate_fn(batch):
    """Custom collate function"""
    pixel_values = torch.stack([item['pixel_values'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    texts = [item['text'] for item in batch]
    
    return {
        'pixel_values': pixel_values,
        'labels': labels,
        'texts': texts,
    }

print("✅ Dataset class and collate_fn defined")

✅ Dataset class and collate_fn defined


In [11]:
# ==================== CONFIG ====================
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_DIR = './thai-trocr-custom-tokenizer'
SP_MODEL_PATH = 'thai_sp_fixed.model'  # ⚠️ แก้ path ตามที่เก็บไฟล์

print(f"🖥️ Device: {DEVICE}")

# ==================== TOKENIZER ====================
thai_tokenizer_fixed = ThaiTokenizerFixed(vocab_file=SP_MODEL_PATH)
print(f"✅ Tokenizer loaded | Vocab size: {thai_tokenizer_fixed.vocab_size}")

# ==================== IMAGE PROCESSOR ====================
image_processor = ViTImageProcessor.from_pretrained('microsoft/trocr-base-handwritten')
print("✅ Image processor loaded")

🖥️ Device: cuda
✅ Tokenizer loaded | Vocab size: 30000
✅ Image processor loaded


In [12]:
print("📦 Loading dataset...")
ds_full = load_dataset("iapp/thai_handwriting_dataset")["train"]
# Split 90/10
split_idx = int(len(ds_full) * 0.9)
train_ds = ds_full.select(range(split_idx))
val_ds = ds_full.select(range(split_idx, len(ds_full)))
print(f"✅ Dataset loaded")
print(f"📊 Train samples: {len(train_ds)}")
print(f"📊 Val samples: {len(val_ds)}")

📦 Loading dataset...
✅ Dataset loaded
📊 Train samples: 12195
📊 Val samples: 1355


In [13]:
# ==================== CREATE PYTORCH DATASETS ====================
print("\n🔄 Creating PyTorch datasets...")
train_dataset = ThaiHandwritingDatasetFixed(train_ds, thai_tokenizer_fixed, image_processor)
val_dataset = ThaiHandwritingDatasetFixed(val_ds, thai_tokenizer_fixed, image_processor)

print(f"✅ Train dataset: {len(train_dataset)} samples")
print(f"✅ Val dataset: {len(val_dataset)} samples")

# ==================== LOAD MODEL ====================
print("\n📥 Loading model architecture...")
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

# Configure for Thai tokenizer
model.config.decoder_start_token_id = thai_tokenizer_fixed.bos_token_id
model.config.pad_token_id = thai_tokenizer_fixed.pad_token_id
model.config.eos_token_id = thai_tokenizer_fixed.eos_token_id
model.config.vocab_size = thai_tokenizer_fixed.vocab_size

# Resize decoder embeddings to match tokenizer
model.decoder.resize_token_embeddings(thai_tokenizer_fixed.vocab_size)

model = model.to(DEVICE)
print(f"✅ Model loaded on {DEVICE}")
print(f"   Vocab size: {model.config.vocab_size}")


🔄 Creating PyTorch datasets...
✅ Train dataset: 12195 samples
✅ Val dataset: 1355 samples

📥 Loading model architecture...


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded on cuda
   Vocab size: 30000


In [22]:
# Cell 6: Test Model - Choose which checkpoint to use
import random
import torch

print("\n" + "="*80)
print("🧪 TESTING MODEL")
print("="*80)

# ⭐ เลือกว่าจะใช้ checkpoint ไหน
USE_CHECKPOINT = "latest"  # "latest", "best", or "current" (model in memory)

if USE_CHECKPOINT == "latest":
    print("\n📥 Loading checkpoint-latest.pt...")
    checkpoint = torch.load('./thai-trocr-custom-tokenizer/checkpoint-latest.pt', map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(DEVICE)
    print(f"✅ Loaded: Epoch {checkpoint['epoch']+1}, Loss {checkpoint['val_loss']:.4f}")

elif USE_CHECKPOINT == "best":
    print("\n📥 Loading best_model.pt...")
    checkpoint = torch.load('./thai-trocr-custom-tokenizer/best_model.pt', map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(DEVICE)
    print(f"✅ Loaded: Epoch {checkpoint['epoch']+1}, Loss {checkpoint['val_loss']:.4f}")

else:  # "current"
    print("\n✅ Using model in memory (current training state)")

print(f"✅ Loaded checkpoint:")
print(f"   Epoch: {checkpoint['epoch'] + 1}")
print(f"   Val Loss: {checkpoint['val_loss']:.4f}")
print(f"   Best Val Loss: {checkpoint.get('best_val_loss', 'N/A'):.4f}")

# Test function
def test_model_samples(model, dataset, tokenizer, image_processor, device, num_samples=10, title="Test Samples"):
    """Test model with random samples and display results"""
    model.eval()
    
    # Random sample indices
    indices = random.sample(range(len(dataset)), num_samples)
    
    print(f"\n{'='*80}")
    print(f"📊 {title}")
    print(f"{'='*80}\n")
    
    results = []
    
    with torch.no_grad():
        for idx, sample_idx in enumerate(indices):
            # Get sample directly from dataset
            sample = dataset[sample_idx]
            
            # Get ground truth
            ground_truth = sample['text']
            
            # Prepare input
            pixel_values = sample['pixel_values'].unsqueeze(0).to(device)
            
            # Generate prediction using Hugging Face generate
            generated_ids = model.generate(
                pixel_values,
                max_length=150,
                num_beams=4,
                early_stopping=True,
                no_repeat_ngram_size=3,
                repetition_penalty=1.5,
                pad_token_id=tokenizer.pad_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
            
            predicted_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            
            # Calculate character-level accuracy
            correct_chars = sum(1 for a, b in zip(predicted_text, ground_truth) if a == b)
            char_accuracy = correct_chars / max(len(ground_truth), 1) * 100
            
            # Store results
            results.append({
                'ground_truth': ground_truth,
                'predicted': predicted_text,
                'accuracy': char_accuracy
            })
            
            # Print results
            print(f"Sample {idx+1}/{num_samples}")
            print(f"├─ Ground Truth: {ground_truth}")
            print(f"├─ Predicted:    {predicted_text}")
            print(f"├─ Char Acc:     {char_accuracy:.1f}%")
            print(f"└─ Match:        {'✅' if ground_truth == predicted_text else '❌'}")
            print()
    
    return results

# Test on train samples
train_results = test_model_samples(
    model=model,
    dataset=train_dataset,
    tokenizer=thai_tokenizer_fixed,
    image_processor=image_processor,
    device=DEVICE,
    num_samples=10,
    title="🎯 TRAIN SET SAMPLES"
)

# Test on validation samples
val_results = test_model_samples(
    model=model,
    dataset=val_dataset,
    tokenizer=thai_tokenizer_fixed,
    image_processor=image_processor,
    device=DEVICE,
    num_samples=10,
    title="🎯 VALIDATION SET SAMPLES"
)

# Calculate overall statistics
print(f"\n{'='*80}")
print("📈 OVERALL STATISTICS")
print(f"{'='*80}")

train_perfect = sum(1 for r in train_results if r['ground_truth'] == r['predicted'])
val_perfect = sum(1 for r in val_results if r['ground_truth'] == r['predicted'])

train_avg_acc = sum(r['accuracy'] for r in train_results) / len(train_results)
val_avg_acc = sum(r['accuracy'] for r in val_results) / len(val_results)

print(f"\n🎯 Train Set:")
print(f"   Perfect matches: {train_perfect}/10 ({train_perfect*10}%)")
print(f"   Avg char accuracy: {train_avg_acc:.2f}%")

print(f"\n🎯 Validation Set:")
print(f"   Perfect matches: {val_perfect}/10 ({val_perfect*10}%)")
print(f"   Avg char accuracy: {val_avg_acc:.2f}%")

print("\n✅ Testing completed!")


🧪 TESTING MODEL

📥 Loading checkpoint-latest.pt...
✅ Loaded: Epoch 100, Loss 0.4900
✅ Loaded checkpoint:
   Epoch: 100
   Val Loss: 0.4900
   Best Val Loss: 0.4880

📊 🎯 TRAIN SET SAMPLES

Sample 1/10
├─ Ground Truth: ต้นทุนในการอยู่ในอำนาจและไม่ให้ถูกจับได้นับวันจะสูงขึ้น
├─ Predicted:    ต้นทุนอยู่ในอยู่ในอำนาจไม่ให้ไม่ให้ไม่ให้ถูกจับนับสูง
├─ Char Acc:     10.9%
└─ Match:        ❌

Sample 2/10
├─ Ground Truth: น่าจะมีอาชีพอื่นมาเสริมรายได้ของครอบครัวเกษตรกรเหล่านี้
├─ Predicted:    น่าจะอาชีพขึ้นมาเสริมรายได้ตอบครอบครัวทางเพศนี้
├─ Char Acc:     9.1%
└─ Match:        ❌

Sample 3/10
├─ Ground Truth: ความพยายามในการอาศัยธนาคารพัฒนาอาเซียกดดันให้มหาวิทยาลัยขับเคลื่อนกระบวนการปฏิรูปประสบความล้มเหลวโดยสิ้นเชิง
├─ Predicted:    ความพยายามในการอาศัยธนาคารเรียกอาเซียทำให้เกิดหรือเพื่อถวายกระบวนการปฏิรูปประสบความล้มเหลวโดยสิ้นเชิง
├─ Char Acc:     31.5%
└─ Match:        ❌

Sample 4/10
├─ Ground Truth: ผลเอกซเรย์พบว่าปอดอักเสบและแพทย์ได้ให้ยาปฏิชีวนะไปแล้ว
├─ Predicted:    ผลเอกซปอดอักเสบและแ

In [23]:
# ============================================================
# Cell: Test on OpenThaiGPT Thai OCR Evaluation Dataset
# ============================================================
from datasets import load_dataset

print("\n" + "="*80)
print("🧪 TESTING ON OPENTHAIGPT THAI-OCR-EVALUATION DATASET")
print("="*80)

# Load evaluation dataset
print("\n📦 Loading evaluation dataset...")
eval_ds = load_dataset("openthaigpt/thai-ocr-evaluation")["test"]
print(f"✅ Loaded {len(eval_ds)} samples")

# Create dataset
eval_dataset = ThaiHandwritingDatasetFixed(eval_ds, thai_tokenizer_fixed, image_processor)

# Test all samples
model.eval()
results = []

print("\n" + "="*80)
print("📊 PREDICTIONS")
print("="*80 + "\n")

with torch.no_grad():
    for i in range(len(eval_dataset)):
        sample = eval_dataset[i]
        ground_truth = sample['text']
        pixel_values = sample['pixel_values'].unsqueeze(0).to(DEVICE)
        
        # Generate prediction
        generated_ids = model.generate(
            pixel_values,
            max_length=150,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
            repetition_penalty=1.5,
            pad_token_id=thai_tokenizer_fixed.pad_token_id,
            bos_token_id=thai_tokenizer_fixed.bos_token_id,
            eos_token_id=thai_tokenizer_fixed.eos_token_id,
        )
        
        predicted = thai_tokenizer_fixed.decode(generated_ids[0], skip_special_tokens=True)
        
        is_correct = ground_truth == predicted
        results.append({
            'ground_truth': ground_truth,
            'predicted': predicted,
            'correct': is_correct
        })
        
        # Print each result
        status = "✅" if is_correct else "❌"
        print(f"[{i+1:3d}/104] {status}")
        print(f"   GT:   {ground_truth}")
        print(f"   Pred: {predicted}")
        print()

# ==================== SUMMARY ====================
print("="*80)
print("📈 SUMMARY")
print("="*80)

correct_count = sum(1 for r in results if r['correct'])
total_count = len(results)
accuracy = correct_count / total_count * 100

print(f"\n🎯 Exact Match: {correct_count}/{total_count} ({accuracy:.2f}%)")

# Character-level accuracy
total_chars = sum(len(r['ground_truth']) for r in results)
correct_chars = sum(
    sum(1 for a, b in zip(r['predicted'], r['ground_truth']) if a == b)
    for r in results
)
char_accuracy = correct_chars / total_chars * 100 if total_chars > 0 else 0

print(f"📝 Character Accuracy: {char_accuracy:.2f}%")

# Show incorrect predictions
incorrect = [r for r in results if not r['correct']]
print(f"\n❌ Incorrect predictions: {len(incorrect)}")

print("\n✅ Evaluation completed!")


🧪 TESTING ON OPENTHAIGPT THAI-OCR-EVALUATION DATASET

📦 Loading evaluation dataset...
✅ Loaded 104 samples

📊 PREDICTIONS

[  1/104] ❌
   GT:   bio-engineer
   Pred: ไอ้--

[  2/104] ❌
   GT:   ทดสอบโมเดล
   Pred: คอมพิวเตอร์วี

[  3/104] ❌
   GT:   ๓๑/๐๑/๒๕๖๗
   Pred: คอมพิวเตอร์มั

[  4/104] ❌
   GT:   31/01/2567
   Pred: รอยนา

[  5/104] ❌
   GT:   3.12 Trillion
   Pred: รอยไทย

[  6/104] ❌
   GT:   #VOO and chill
   Pred: ท่าสอดคล้อง่อ่ว

[  7/104] ❌
   GT:   Intel Corp
   Pred: ฑออกศ

[  8/104] ❌
   GT:   AMD INC.
   Pred: ลอบย

[  9/104] ❌
   GT:   NVIDIA
   Pred: การเจรจา

[ 10/104] ❌
   GT:   Alphabet
   Pred: กังอะไร

[ 11/104] ❌
   GT:   MasterCard
   Pred: ให้คลองวี

[ 12/104] ❌
   GT:   META PLATFORMS, INC.
   Pred: (ตรงกันวิทยาศาสตร์วา จำกัด

[ 13/104] ❌
   GT:   ELI LILLY
   Pred: (ประโยคประโยค

[ 14/104] ❌
   GT:   Arthur J. Gallagher
   Pred: ท่าออกมาร์จางใจยดู

[ 15/104] ❌
   GT:   พินัยกรรม
   Pred: ยโสธร

[ 16/104] ❌
   GT:   ฝากคุณยาย
   Pred: พัทลุง

[ 17/104] ❌
 

In [26]:
# ============================================================
# Cell: Test on bypkt/thai_handwritten_datasets (Random 100 samples)
# ============================================================
from datasets import load_dataset
import random

def calculate_cer(predicted: str, ground_truth: str) -> float:
    """Calculate Character Error Rate using Levenshtein distance"""
    def levenshtein(s1, s2):
        if len(s1) < len(s2):
            return levenshtein(s2, s1)
        if len(s2) == 0:
            return len(s1)
        
        prev_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            curr_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = prev_row[j + 1] + 1
                deletions = curr_row[j] + 1
                substitutions = prev_row[j] + (c1 != c2)
                curr_row.append(min(insertions, deletions, substitutions))
            prev_row = curr_row
        return prev_row[-1]
    
    if len(ground_truth) == 0:
        return 1.0 if len(predicted) > 0 else 0.0
    
    return levenshtein(predicted, ground_truth) / len(ground_truth)


print("\n" + "="*80)
print("🧪 TESTING ON BYPKT/THAI_HANDWRITTEN_DATASETS")
print("="*80)

# Load dataset
print("\n📦 Loading dataset...")
bypkt_ds = load_dataset("bypkt/thai_handwritten_datasets")["train"]
print(f"✅ Loaded {len(bypkt_ds)} samples")

# Random sample 100
random.seed(42)
sample_indices = random.sample(range(len(bypkt_ds)), 100)
print(f"🎲 Randomly selected 100 samples")

# Create dataset
bypkt_dataset = ThaiHandwritingDatasetFixed(bypkt_ds, thai_tokenizer_fixed, image_processor)

# Test samples
model.eval()
results = []

print("\n" + "="*80)
print("📊 PREDICTIONS (100 random samples)")
print("="*80 + "\n")

with torch.no_grad():
    for i, idx in enumerate(sample_indices):
        sample = bypkt_dataset[idx]
        ground_truth = sample['text']
        pixel_values = sample['pixel_values'].unsqueeze(0).to(DEVICE)
        
        # Generate prediction
        generated_ids = model.generate(
            pixel_values,
            max_length=150,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
            repetition_penalty=1.5,
            pad_token_id=thai_tokenizer_fixed.pad_token_id,
            bos_token_id=thai_tokenizer_fixed.bos_token_id,
            eos_token_id=thai_tokenizer_fixed.eos_token_id,
        )
        
        predicted = thai_tokenizer_fixed.decode(generated_ids[0], skip_special_tokens=True)
        
        # Calculate CER
        cer = calculate_cer(predicted, ground_truth)
        is_correct = ground_truth == predicted
        
        results.append({
            'index': idx,
            'ground_truth': ground_truth,
            'predicted': predicted,
            'correct': is_correct,
            'cer': cer
        })
        
        # Print each result
        status = "✅" if is_correct else "❌"
        print(f"[{i+1:3d}/100] (idx:{idx:5d}) {status} | CER: {cer:.4f}")
        print(f"   GT:   {ground_truth}")
        print(f"   Pred: {predicted}")
        print()

# ==================== SUMMARY ====================
print("="*80)
print("📈 SUMMARY")
print("="*80)

correct_count = sum(1 for r in results if r['correct'])
total_count = len(results)
accuracy = correct_count / total_count * 100

print(f"\n🎯 Exact Match: {correct_count}/{total_count} ({accuracy:.2f}%)")

# Average CER
avg_cer = sum(r['cer'] for r in results) / len(results)
print(f"📊 Average CER: {avg_cer:.4f} ({avg_cer*100:.2f}%)")

# CER breakdown
cer_perfect = sum(1 for r in results if r['cer'] == 0)
cer_good = sum(1 for r in results if 0 < r['cer'] <= 0.1)
cer_ok = sum(1 for r in results if 0.1 < r['cer'] <= 0.3)
cer_bad = sum(1 for r in results if 0.3 < r['cer'] <= 1.0)
cer_worse = sum(1 for r in results if r['cer'] > 1.0)

print(f"\n📊 CER Distribution:")
print(f"   CER = 0 (Perfect):     {cer_perfect:3d} ({cer_perfect/total_count*100:5.1f}%)")
print(f"   CER ≤ 0.1 (Excellent): {cer_good:3d} ({cer_good/total_count*100:5.1f}%)")
print(f"   CER ≤ 0.3 (Good):      {cer_ok:3d} ({cer_ok/total_count*100:5.1f}%)")
print(f"   CER ≤ 1.0 (Poor):      {cer_bad:3d} ({cer_bad/total_count*100:5.1f}%)")
print(f"   CER > 1.0 (Very Poor): {cer_worse:3d} ({cer_worse/total_count*100:5.1f}%)")

print(f"\n✅ Correct: {correct_count} | ❌ Incorrect: {total_count - correct_count}")
print("\n✅ Evaluation completed!")


🧪 TESTING ON BYPKT/THAI_HANDWRITTEN_DATASETS

📦 Loading dataset...
✅ Loaded 10000 samples
🎲 Randomly selected 100 samples

📊 PREDICTIONS (100 random samples)

[  1/100] (idx: 1824) ❌ | CER: 0.4938
   GT:   ถ้าขาดความเข้าใจหรือขาดความระมัดระวังแล้วหน้าดินก็จะถูกน้ำพัดพาไปจนเหลือแต่ดินดาน
   Pred: ถ้าขาดความเข้าใจหรือขาดนำสำเร็จแล้วจะถูกจะถูกพัดไปไปไปอย่างชัดเจน

[  2/100] (idx:  409) ❌ | CER: 0.3810
   GT:   ...และเห็นภาพของหญิงสาวผู้หนึ่งยืนนิ่งสงบอยู่ระหว่างช่องประตูหน้าบ้านที่เปิดกว้าง...
   Pred: ...และภาพของหญิงสาวหญิงสาวผู้หนึ่งยืนนิ่งมอยู่ระหว่างช่องบ้านที่ระเบิดที่เปิดกว้างกว้าง...

[  3/100] (idx: 4506) ❌ | CER: 0.4865
   GT:   นัยว่าคนไทยมาตั้งสาขาที่นี่ใช่ไหมคะ?"
   Pred: นัยมาตั้งที่นี่ใช่ใช่?"?

[  4/100] (idx: 4012) ❌ | CER: 0.3253
   GT:   อาจมีการสั่งทำเครื่องถ้วยเบญจรงค์และลายน้ำทองจากญี่ปุ่นสืบต่อจากสมัยอยุธยาก็เป็นได้
   Pred: อาจมีการสั่งเครื่องถ้วยสมบูรณ์และจากทองมากกว่าในเรื่องต่อจากสมัยอยุธยาก็เป็น

[  5/100] (idx: 3657) ❌ | CER: 0.3200
   GT:   จะมีการแถลงผลงานข

In [28]:
# ============================================================
# Cell: Compare Your Model vs OpenThaiGPT Thai-TrOCR
# ============================================================
from datasets import load_dataset
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import random
import torch

def calculate_cer(predicted: str, ground_truth: str) -> float:
    """Calculate Character Error Rate using Levenshtein distance"""
    def levenshtein(s1, s2):
        if len(s1) < len(s2):
            return levenshtein(s2, s1)
        if len(s2) == 0:
            return len(s1)
        
        prev_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            curr_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = prev_row[j + 1] + 1
                deletions = curr_row[j] + 1
                substitutions = prev_row[j] + (c1 != c2)
                curr_row.append(min(insertions, deletions, substitutions))
            prev_row = curr_row
        return prev_row[-1]
    
    if len(ground_truth) == 0:
        return 1.0 if len(predicted) > 0 else 0.0
    
    return levenshtein(predicted, ground_truth) / len(ground_truth)


print("\n" + "="*80)
print("🏆 MODEL COMPARISON: Your Model vs OpenThaiGPT Thai-TrOCR")
print("="*80)

# ==================== LOAD OPENTHAIGPT MODEL ====================
print("\n📦 Loading OpenThaiGPT Thai-TrOCR model...")
openthaigpt_processor = TrOCRProcessor.from_pretrained('openthaigpt/thai-trocr')
openthaigpt_model = VisionEncoderDecoderModel.from_pretrained('openthaigpt/thai-trocr')
openthaigpt_model = openthaigpt_model.to(DEVICE)
openthaigpt_model.eval()
print("✅ OpenThaiGPT model loaded")

# ==================== LOAD TEST DATASET ====================
print("\n📦 Loading test dataset (bypkt/thai_handwritten_datasets)...")
bypkt_ds = load_dataset("bypkt/thai_handwritten_datasets")["train"]
print(f"✅ Loaded {len(bypkt_ds)} samples")

# Random sample
random.seed(42)
sample_indices = random.sample(range(len(bypkt_ds)), 200)
print(f"🎲 Randomly selected 50 samples for comparison")

# Create dataset for your model
bypkt_dataset = ThaiHandwritingDatasetFixed(bypkt_ds, thai_tokenizer_fixed, image_processor)

# ==================== RUN COMPARISON ====================
print("\n" + "="*80)
print("📊 PREDICTIONS COMPARISON")
print("="*80 + "\n")

your_results = []
openthaigpt_results = []

with torch.no_grad():
    for i, idx in enumerate(sample_indices):
        # Get raw data for OpenThaiGPT
        raw_item = bypkt_ds[idx]
        ground_truth = raw_item['text']
        
        # Process image for OpenThaiGPT
        image = raw_item['image']
        if not isinstance(image, Image.Image):
            image = Image.open(io.BytesIO(image['bytes']))
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # ===== YOUR MODEL =====
        sample = bypkt_dataset[idx]
        pixel_values_yours = sample['pixel_values'].unsqueeze(0).to(DEVICE)
        
        generated_ids_yours = model.generate(
            pixel_values_yours,
            max_length=150,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
            repetition_penalty=1.5,
            pad_token_id=thai_tokenizer_fixed.pad_token_id,
            bos_token_id=thai_tokenizer_fixed.bos_token_id,
            eos_token_id=thai_tokenizer_fixed.eos_token_id,
        )
        pred_yours = thai_tokenizer_fixed.decode(generated_ids_yours[0], skip_special_tokens=True)
        cer_yours = calculate_cer(pred_yours, ground_truth)
        
        # ===== OPENTHAIGPT MODEL =====
        pixel_values_otg = openthaigpt_processor(images=image, return_tensors="pt").pixel_values.to(DEVICE)
        
        generated_ids_otg = openthaigpt_model.generate(pixel_values_otg)
        pred_otg = openthaigpt_processor.batch_decode(generated_ids_otg, skip_special_tokens=True)[0]
        cer_otg = calculate_cer(pred_otg, ground_truth)
        
        # Store results
        your_results.append({'gt': ground_truth, 'pred': pred_yours, 'cer': cer_yours})
        openthaigpt_results.append({'gt': ground_truth, 'pred': pred_otg, 'cer': cer_otg})
        
        # Determine winner
        if cer_yours < cer_otg:
            winner = "🏆 YOU"
        elif cer_otg < cer_yours:
            winner = "🏆 OTG"
        else:
            winner = "🤝 TIE"
        
        # Print comparison
        print(f"[{i+1:2d}/200] {winner}")
        print(f"   GT:       {ground_truth}")
        print(f"   Yours:    {pred_yours} (CER: {cer_yours:.4f})")
        print(f"   ThaiTrOCR: {pred_otg} (CER: {cer_otg:.4f})")
        print()

# ==================== SUMMARY ====================
print("="*80)
print("📈 COMPARISON SUMMARY")
print("="*80)

# Your model stats
your_avg_cer = sum(r['cer'] for r in your_results) / len(your_results)
your_exact = sum(1 for r in your_results if r['cer'] == 0)

# OpenThaiGPT stats  
otg_avg_cer = sum(r['cer'] for r in openthaigpt_results) / len(openthaigpt_results)
otg_exact = sum(1 for r in openthaigpt_results if r['cer'] == 0)

# Win counts
your_wins = sum(1 for y, o in zip(your_results, openthaigpt_results) if y['cer'] < o['cer'])
otg_wins = sum(1 for y, o in zip(your_results, openthaigpt_results) if o['cer'] < y['cer'])
ties = sum(1 for y, o in zip(your_results, openthaigpt_results) if y['cer'] == o['cer'])

print(f"\n{'Metric':<25} {'Your Model':>15} {'OpenThaiGPT':>15}")
print("-"*55)
print(f"{'Average CER':<25} {your_avg_cer:>15.4f} {otg_avg_cer:>15.4f}")
print(f"{'Exact Match':<25} {your_exact:>15} {otg_exact:>15}")
print(f"{'Wins':<25} {your_wins:>15} {otg_wins:>15}")
print(f"{'Ties':<25} {ties:>15}")

print(f"\n🏆 WINNER: ", end="")
if your_avg_cer < otg_avg_cer:
    print(f"YOUR MODEL! (CER {your_avg_cer:.4f} vs {otg_avg_cer:.4f})")
elif otg_avg_cer < your_avg_cer:
    print(f"OpenThaiGPT Thai-TrOCR (CER {otg_avg_cer:.4f} vs {your_avg_cer:.4f})")
else:
    print("TIE!")

print("\n✅ Comparison completed!")


🏆 MODEL COMPARISON: Your Model vs OpenThaiGPT Thai-TrOCR

📦 Loading OpenThaiGPT Thai-TrOCR model...
✅ OpenThaiGPT model loaded

📦 Loading test dataset (bypkt/thai_handwritten_datasets)...
✅ Loaded 10000 samples
🎲 Randomly selected 50 samples for comparison

📊 PREDICTIONS COMPARISON

[ 1/50] 🏆 YOU
   GT:       ถ้าขาดความเข้าใจหรือขาดความระมัดระวังแล้วหน้าดินก็จะถูกน้ำพัดพาไปจนเหลือแต่ดินดาน
   Yours:    ถ้าขาดความเข้าใจหรือขาดนำสำเร็จแล้วจะถูกจะถูกพัดไปไปไปอย่างชัดเจน (CER: 0.4938)
   ThaiTrOCR: พันต้นน้ํานนเธียเรียลพัฒนาปมละป (CER: 0.8765)

[ 2/50] 🏆 YOU
   GT:       ...และเห็นภาพของหญิงสาวผู้หนึ่งยืนนิ่งสงบอยู่ระหว่างช่องประตูหน้าบ้านที่เปิดกว้าง...
   Yours:    ...และภาพของหญิงสาวหญิงสาวผู้หนึ่งยืนนิ่งมอยู่ระหว่างช่องบ้านที่ระเบิดที่เปิดกว้างกว้าง... (CER: 0.3810)
   ThaiTrOCR: มองพพนนพระนางคน (CER: 0.8929)

[ 3/50] 🏆 YOU
   GT:       นัยว่าคนไทยมาตั้งสาขาที่นี่ใช่ไหมคะ?"
   Yours:    นัยมาตั้งที่นี่ใช่ใช่?"? (CER: 0.4865)
   ThaiTrOCR: พันเอก อ่านพรหม๕ททนับB(นwน น" (CER: 0.8649)

[

In [38]:
# ============================================================
# Cell: Compare 4 Models (with SSL fix)
# ============================================================
from datasets import load_dataset
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import random
import torch
import numpy as np
from PIL import Image
import io

# Fix SSL issue
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def calculate_cer(predicted: str, ground_truth: str) -> float:
    """Calculate Character Error Rate using Levenshtein distance"""
    def levenshtein(s1, s2):
        if len(s1) < len(s2):
            return levenshtein(s2, s1)
        if len(s2) == 0:
            return len(s1)
        
        prev_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            curr_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = prev_row[j + 1] + 1
                deletions = curr_row[j] + 1
                substitutions = prev_row[j] + (c1 != c2)
                curr_row.append(min(insertions, deletions, substitutions))
            prev_row = curr_row
        return prev_row[-1]
    
    if len(ground_truth) == 0:
        return 1.0 if len(predicted) > 0 else 0.0
    
    return levenshtein(predicted, ground_truth) / len(ground_truth)


print("\n" + "="*80)
print("🏆 4-WAY MODEL COMPARISON")
print("   Your Model vs OpenThaiGPT vs EasyOCR vs Tesseract")
print("="*80)

# ==================== LOAD MODELS ====================
print("\n📦 Loading models...")

# 1. OpenThaiGPT Thai-TrOCR
print("   Loading OpenThaiGPT Thai-TrOCR...")
openthaigpt_processor = TrOCRProcessor.from_pretrained('openthaigpt/thai-trocr')
openthaigpt_model = VisionEncoderDecoderModel.from_pretrained('openthaigpt/thai-trocr')
openthaigpt_model = openthaigpt_model.to(DEVICE)
openthaigpt_model.eval()
print("   ✅ OpenThaiGPT loaded")

# 2. EasyOCR
print("   Loading EasyOCR...")
try:
    import easyocr
    easyocr_reader = easyocr.Reader(['th', 'en'], gpu=torch.cuda.is_available())
    easyocr_available = True
    print("   ✅ EasyOCR loaded")
except Exception as e:
    print(f"   ⚠️ EasyOCR failed: {e}")
    easyocr_available = False

# 3. Tesseract
print("   Loading Tesseract...")
try:
    import pytesseract
    # Test if tesseract works
    pytesseract.get_tesseract_version()
    tesseract_available = True
    print("   ✅ Tesseract loaded")
except Exception as e:
    print(f"   ⚠️ Tesseract failed: {e}")
    tesseract_available = False

print(f"\n✅ Models ready: Your Model, OpenThaiGPT" + 
      (", EasyOCR" if easyocr_available else "") + 
      (", Tesseract" if tesseract_available else ""))

# ==================== LOAD TEST DATASET ====================
print("\n📦 Loading test dataset (bypkt/thai_handwritten_datasets)...")
bypkt_ds = load_dataset("bypkt/thai_handwritten_datasets")["train"]
print(f"✅ Loaded {len(bypkt_ds)} samples")

# Random sample
random.seed(42)
sample_indices = random.sample(range(len(bypkt_ds)), 300)
print(f"🎲 Randomly selected 50 samples for comparison")

# Create dataset for your model
bypkt_dataset = ThaiHandwritingDatasetFixed(bypkt_ds, thai_tokenizer_fixed, image_processor)

# ==================== RUN COMPARISON ====================
print("\n" + "="*80)
print("📊 PREDICTIONS COMPARISON (50 samples)")
print("="*80 + "\n")

results = {
    'yours': [],
    'openthaigpt': [],
    'easyocr': [],
    'tesseract': []
}

with torch.no_grad():
    for i, idx in enumerate(sample_indices):
        # Get raw data
        raw_item = bypkt_ds[idx]
        ground_truth = raw_item['text']
        
        # Process image
        image = raw_item['image']
        if not isinstance(image, Image.Image):
            image = Image.open(io.BytesIO(image['bytes']))
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # ===== 1. YOUR MODEL =====
        sample = bypkt_dataset[idx]
        pixel_values_yours = sample['pixel_values'].unsqueeze(0).to(DEVICE)
        
        generated_ids_yours = model.generate(
            pixel_values_yours,
            max_length=150,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
            repetition_penalty=1.5,
            pad_token_id=thai_tokenizer_fixed.pad_token_id,
            bos_token_id=thai_tokenizer_fixed.bos_token_id,
            eos_token_id=thai_tokenizer_fixed.eos_token_id,
        )
        pred_yours = thai_tokenizer_fixed.decode(generated_ids_yours[0], skip_special_tokens=True)
        cer_yours = calculate_cer(pred_yours, ground_truth)
        
        # ===== 2. OPENTHAIGPT MODEL =====
        pixel_values_otg = openthaigpt_processor(images=image, return_tensors="pt").pixel_values.to(DEVICE)
        generated_ids_otg = openthaigpt_model.generate(pixel_values_otg)
        pred_otg = openthaigpt_processor.batch_decode(generated_ids_otg, skip_special_tokens=True)[0]
        cer_otg = calculate_cer(pred_otg, ground_truth)
        
        # ===== 3. EASYOCR =====
        if easyocr_available:
            try:
                easyocr_result = easyocr_reader.readtext(np.array(image), detail=0)
                pred_easyocr = ''.join(easyocr_result) if easyocr_result else ''
            except:
                pred_easyocr = ''
        else:
            pred_easyocr = '[N/A]'
        cer_easyocr = calculate_cer(pred_easyocr, ground_truth) if easyocr_available else -1
        
        # ===== 4. TESSERACT =====
        if tesseract_available:
            try:
                pred_tesseract = pytesseract.image_to_string(image, lang='tha+eng').strip()
                pred_tesseract = pred_tesseract.replace('\n', ' ').replace('  ', ' ')
            except:
                pred_tesseract = ''
        else:
            pred_tesseract = '[N/A]'
        cer_tesseract = calculate_cer(pred_tesseract, ground_truth) if tesseract_available else -1
        
        # Store results
        results['yours'].append({'gt': ground_truth, 'pred': pred_yours, 'cer': cer_yours})
        results['openthaigpt'].append({'gt': ground_truth, 'pred': pred_otg, 'cer': cer_otg})
        results['easyocr'].append({'gt': ground_truth, 'pred': pred_easyocr, 'cer': cer_easyocr})
        results['tesseract'].append({'gt': ground_truth, 'pred': pred_tesseract, 'cer': cer_tesseract})
        
        # Find winner (only from available models)
        cers = {'Yours': cer_yours, 'OTG': cer_otg}
        if easyocr_available:
            cers['Easy'] = cer_easyocr
        if tesseract_available:
            cers['Tess'] = cer_tesseract
        winner = min(cers, key=cers.get)
        
        # Print comparison
        print(f"[{i+1:2d}/300] 🏆 Winner: {winner}")
        print(f"   GT:        {ground_truth}")
        print(f"   Yours:     {pred_yours[:60]}{'...' if len(pred_yours) > 60 else ''} (CER: {cer_yours:.4f})")
        print(f"   ThaiTrOCR: {pred_otg[:60]}{'...' if len(pred_otg) > 60 else ''} (CER: {cer_otg:.4f})")
        if easyocr_available:
            print(f"   EasyOCR:   {pred_easyocr[:60]}{'...' if len(pred_easyocr) > 60 else ''} (CER: {cer_easyocr:.4f})")
        if tesseract_available:
            print(f"   Tesseract: {pred_tesseract[:60]}{'...' if len(pred_tesseract) > 60 else ''} (CER: {cer_tesseract:.4f})")
        print()

# ==================== SUMMARY ====================
print("="*80)
print("📈 FINAL COMPARISON SUMMARY")
print("="*80)

# Calculate stats for each model
model_names = {'yours': 'Your Model', 'openthaigpt': 'OpenThaiGPT', 'easyocr': 'EasyOCR', 'tesseract': 'Tesseract'}
active_models = ['yours', 'openthaigpt']
if easyocr_available:
    active_models.append('easyocr')
if tesseract_available:
    active_models.append('tesseract')

stats = {}
for name in active_models:
    res = results[name]
    valid_res = [r for r in res if r['cer'] >= 0]
    avg_cer = sum(r['cer'] for r in valid_res) / len(valid_res) if valid_res else -1
    exact_match = sum(1 for r in valid_res if r['cer'] == 0)
    stats[name] = {'avg_cer': avg_cer, 'exact_match': exact_match}

# Count wins
win_counts = {name: 0 for name in active_models}
for i in range(len(sample_indices)):
    cers = {name: results[name][i]['cer'] for name in active_models if results[name][i]['cer'] >= 0}
    if cers:
        winner = min(cers, key=cers.get)
        win_counts[winner] += 1

# Print table
print(f"\n{'Model':<20} {'Avg CER':>12} {'Exact Match':>15} {'Wins':>10}")
print("-"*60)
for name in active_models:
    print(f"{model_names[name]:<20} {stats[name]['avg_cer']:>12.4f} {stats[name]['exact_match']:>15} {win_counts[name]:>10}")

# Overall winner
best_model = min(active_models, key=lambda x: stats[x]['avg_cer'])
print(f"\n🏆 OVERALL WINNER: {model_names[best_model]} (Avg CER: {stats[best_model]['avg_cer']:.4f})")

# CER Distribution
print(f"\n📊 CER Distribution:")
print(f"{'Model':<20} {'CER=0':>8} {'≤0.1':>8} {'≤0.3':>8} {'≤1.0':>8} {'>1.0':>8}")
print("-"*60)

for name in active_models:
    res = [r for r in results[name] if r['cer'] >= 0]
    cer_0 = sum(1 for r in res if r['cer'] == 0)
    cer_01 = sum(1 for r in res if 0 < r['cer'] <= 0.1)
    cer_03 = sum(1 for r in res if 0.1 < r['cer'] <= 0.3)
    cer_10 = sum(1 for r in res if 0.3 < r['cer'] <= 1.0)
    cer_over = sum(1 for r in res if r['cer'] > 1.0)
    print(f"{model_names[name]:<20} {cer_0:>8} {cer_01:>8} {cer_03:>8} {cer_10:>8} {cer_over:>8}")

print("\n✅ Comparison completed!")


🏆 4-WAY MODEL COMPARISON
   Your Model vs OpenThaiGPT vs EasyOCR vs Tesseract

📦 Loading models...
   Loading OpenThaiGPT Thai-TrOCR...
   ✅ OpenThaiGPT loaded
   Loading EasyOCR...
   ✅ EasyOCR loaded
   Loading Tesseract...
   ✅ Tesseract loaded

✅ Models ready: Your Model, OpenThaiGPT, EasyOCR, Tesseract

📦 Loading test dataset (bypkt/thai_handwritten_datasets)...
✅ Loaded 10000 samples
🎲 Randomly selected 50 samples for comparison

📊 PREDICTIONS COMPARISON (50 samples)

[ 1/300] 🏆 Winner: Easy
   GT:        ถ้าขาดความเข้าใจหรือขาดความระมัดระวังแล้วหน้าดินก็จะถูกน้ำพัดพาไปจนเหลือแต่ดินดาน
   Yours:     ถ้าขาดความเข้าใจหรือขาดนำสำเร็จแล้วจะถูกจะถูกพัดไปไปไปอย่างช... (CER: 0.4938)
   ThaiTrOCR: พันต้นน้ํานนเธียเรียลพัฒนาปมละป (CER: 0.8765)
   EasyOCR:   กาบาดความเข้ำคหรือ ขาดภาามระมัดระจัง แล็ำหน้าดินกิจะกานามัดพ... (CER: 0.3457)
   Tesseract: ล  7   สนจษี ส   ¢ | : ana Fae ANN มัดระจัง เพณ์ตินก จนเชถี๊... (CER: 0.8642)

[ 2/300] 🏆 Winner: Yours
   GT:        ...และเห็นภาพของหญิงสาวผ

In [46]:
# ============================================================
# Cell: Compare 4 Models with Better Metrics
# ============================================================
from datasets import load_dataset
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import random
import torch
import numpy as np
from PIL import Image
import io
import re

# Fix SSL issue
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# ==================== METRICS ====================

def calculate_cer(predicted: str, ground_truth: str) -> float:
    """CER - Character Error Rate"""
    def levenshtein(s1, s2):
        if len(s1) < len(s2):
            return levenshtein(s2, s1)
        if len(s2) == 0:
            return len(s1)
        prev_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            curr_row = [i + 1]
            for j, c2 in enumerate(s2):
                curr_row.append(min(prev_row[j + 1] + 1, curr_row[j] + 1, prev_row[j] + (c1 != c2)))
            prev_row = curr_row
        return prev_row[-1]
    
    if len(ground_truth) == 0:
        return 1.0 if len(predicted) > 0 else 0.0
    return levenshtein(predicted, ground_truth) / len(ground_truth)


def calculate_char_accuracy(predicted: str, ground_truth: str) -> float:
    """Character Accuracy - % ตัวอักษรที่ตรงกัน (position-independent)"""
    if len(ground_truth) == 0:
        return 1.0 if len(predicted) == 0 else 0.0
    
    # นับตัวอักษรที่มีใน ground truth
    from collections import Counter
    gt_chars = Counter(ground_truth)
    pred_chars = Counter(predicted)
    
    # หาตัวที่ตรงกัน
    matches = sum((gt_chars & pred_chars).values())
    return matches / len(ground_truth)


def calculate_word_accuracy(predicted: str, ground_truth: str) -> float:
    """Word Accuracy - % คำที่ปรากฏใน ground truth"""
    # ตัดคำแบบง่าย (ใช้ space + Thai word boundary)
    def tokenize(text):
        # แยกด้วย space และ punctuation
        words = re.findall(r'[\u0E00-\u0E7F]+|[a-zA-Z]+|[0-9]+', text)
        return set(words)
    
    gt_words = tokenize(ground_truth)
    pred_words = tokenize(predicted)
    
    if len(gt_words) == 0:
        return 1.0 if len(pred_words) == 0 else 0.0
    
    matches = len(gt_words & pred_words)
    return matches / len(gt_words)


def calculate_ngram_precision(predicted: str, ground_truth: str, n: int = 3) -> float:
    """N-gram Precision - % ของ n-gram ที่ถูกต้อง"""
    def get_ngrams(text, n):
        return set(text[i:i+n] for i in range(len(text) - n + 1))
    
    if len(ground_truth) < n:
        return 1.0 if predicted == ground_truth else 0.0
    
    gt_ngrams = get_ngrams(ground_truth, n)
    pred_ngrams = get_ngrams(predicted, n)
    
    if len(pred_ngrams) == 0:
        return 0.0
    
    matches = len(gt_ngrams & pred_ngrams)
    return matches / len(pred_ngrams)


def calculate_sequence_match(predicted: str, ground_truth: str) -> float:
    """Longest Common Subsequence Ratio - วัดลำดับที่ถูกต้อง"""
    def lcs_length(s1, s2):
        m, n = len(s1), len(s2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if s1[i-1] == s2[j-1]:
                    dp[i][j] = dp[i-1][j-1] + 1
                else:
                    dp[i][j] = max(dp[i-1][j], dp[i][j-1])
        return dp[m][n]
    
    if len(ground_truth) == 0:
        return 1.0 if len(predicted) == 0 else 0.0
    
    lcs = lcs_length(predicted, ground_truth)
    return lcs / len(ground_truth)


def calculate_all_metrics(predicted: str, ground_truth: str) -> dict:
    """คำนวณทุก metrics"""
    return {
        'cer': calculate_cer(predicted, ground_truth),
        'char_acc': calculate_char_accuracy(predicted, ground_truth),
        'word_acc': calculate_word_accuracy(predicted, ground_truth),
        'ngram3': calculate_ngram_precision(predicted, ground_truth, 3),
        'seq_match': calculate_sequence_match(predicted, ground_truth),
    }


def calculate_combined_score(metrics: dict) -> float:
    """Combined Score - รวมทุก metric (higher is better)"""
    # CER ต้อง invert เพราะ lower is better
    cer_score = max(0, 1 - metrics['cer'])
    
    return (
        cer_score * 0.2 +           # CER (inverted)
        metrics['char_acc'] * 0.2 +  # Character Accuracy
        metrics['word_acc'] * 0.25 + # Word Accuracy (สำคัญ!)
        metrics['ngram3'] * 0.15 +   # 3-gram Precision
        metrics['seq_match'] * 0.2   # Sequence Match
    )


print("\n" + "="*80)
print("🏆 4-WAY MODEL COMPARISON (Multi-Metric)")
print("   Your Model vs OpenThaiGPT vs EasyOCR vs Tesseract")
print("="*80)

# ==================== LOAD MODELS ====================
print("\n📦 Loading models...")

# 1. OpenThaiGPT
print("   Loading OpenThaiGPT Thai-TrOCR...")
openthaigpt_processor = TrOCRProcessor.from_pretrained('openthaigpt/thai-trocr')
openthaigpt_model = VisionEncoderDecoderModel.from_pretrained('openthaigpt/thai-trocr')
openthaigpt_model = openthaigpt_model.to(DEVICE)
openthaigpt_model.eval()
print("   ✅ OpenThaiGPT loaded")

# 2. EasyOCR
print("   Loading EasyOCR...")
try:
    import easyocr
    easyocr_reader = easyocr.Reader(['th', 'en'], gpu=torch.cuda.is_available())
    easyocr_available = True
    print("   ✅ EasyOCR loaded")
except Exception as e:
    print(f"   ⚠️ EasyOCR failed: {e}")
    easyocr_available = False

# 3. Tesseract
print("   Loading Tesseract...")
try:
    import pytesseract
    pytesseract.get_tesseract_version()
    tesseract_available = True
    print("   ✅ Tesseract loaded")
except Exception as e:
    print(f"   ⚠️ Tesseract failed: {e}")
    tesseract_available = False

# ==================== LOAD TEST DATASET ====================
print("\n📦 Loading test dataset...")
bypkt_ds = load_dataset("bypkt/thai_handwritten_datasets")["train"]
print(f"✅ Loaded {len(bypkt_ds)} samples")

random.seed(42)
sample_indices = random.sample(range(len(bypkt_ds)), 300)
print(f"🎲 Randomly selected 100 samples")

bypkt_dataset = ThaiHandwritingDatasetFixed(bypkt_ds, thai_tokenizer_fixed, image_processor)

# ==================== RUN COMPARISON ====================
print("\n" + "="*80)
print("📊 PREDICTIONS COMPARISON")
print("="*80 + "\n")

results = {'yours': [], 'openthaigpt': [], 'easyocr': [], 'tesseract': []}

with torch.no_grad():
    for i, idx in enumerate(sample_indices):
        raw_item = bypkt_ds[idx]
        ground_truth = raw_item['text']
        
        image = raw_item['image']
        if not isinstance(image, Image.Image):
            image = Image.open(io.BytesIO(image['bytes']))
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # 1. YOUR MODEL
        sample = bypkt_dataset[idx]
        pixel_values_yours = sample['pixel_values'].unsqueeze(0).to(DEVICE)
        generated_ids_yours = model.generate(
            pixel_values_yours, max_length=150, num_beams=4, early_stopping=True,
            no_repeat_ngram_size=3, repetition_penalty=1.5,
            pad_token_id=thai_tokenizer_fixed.pad_token_id,
            bos_token_id=thai_tokenizer_fixed.bos_token_id,
            eos_token_id=thai_tokenizer_fixed.eos_token_id,
        )
        pred_yours = thai_tokenizer_fixed.decode(generated_ids_yours[0], skip_special_tokens=True)
        metrics_yours = calculate_all_metrics(pred_yours, ground_truth)
        metrics_yours['combined'] = calculate_combined_score(metrics_yours)
        
        # 2. OPENTHAIGPT
        pixel_values_otg = openthaigpt_processor(images=image, return_tensors="pt").pixel_values.to(DEVICE)
        generated_ids_otg = openthaigpt_model.generate(pixel_values_otg)
        pred_otg = openthaigpt_processor.batch_decode(generated_ids_otg, skip_special_tokens=True)[0]
        metrics_otg = calculate_all_metrics(pred_otg, ground_truth)
        metrics_otg['combined'] = calculate_combined_score(metrics_otg)
        
        # 3. EASYOCR
        if easyocr_available:
            try:
                easyocr_result = easyocr_reader.readtext(np.array(image), detail=0)
                pred_easyocr = ''.join(easyocr_result) if easyocr_result else ''
            except:
                pred_easyocr = ''
            metrics_easyocr = calculate_all_metrics(pred_easyocr, ground_truth)
            metrics_easyocr['combined'] = calculate_combined_score(metrics_easyocr)
        else:
            pred_easyocr = '[N/A]'
            metrics_easyocr = {'cer': -1, 'combined': -1}
        
        # 4. TESSERACT
        if tesseract_available:
            try:
                pred_tesseract = pytesseract.image_to_string(image, lang='tha+eng').strip()
                pred_tesseract = pred_tesseract.replace('\n', ' ').replace('  ', ' ')
            except:
                pred_tesseract = ''
            metrics_tesseract = calculate_all_metrics(pred_tesseract, ground_truth)
            metrics_tesseract['combined'] = calculate_combined_score(metrics_tesseract)
        else:
            pred_tesseract = '[N/A]'
            metrics_tesseract = {'cer': -1, 'combined': -1}
        
        # Store
        results['yours'].append({'gt': ground_truth, 'pred': pred_yours, **metrics_yours})
        results['openthaigpt'].append({'gt': ground_truth, 'pred': pred_otg, **metrics_otg})
        results['easyocr'].append({'gt': ground_truth, 'pred': pred_easyocr, **metrics_easyocr})
        results['tesseract'].append({'gt': ground_truth, 'pred': pred_tesseract, **metrics_tesseract})
        
        # Find winner by combined score
        scores = {'Yours': metrics_yours['combined'], 'OTG': metrics_otg['combined']}
        if easyocr_available:
            scores['Easy'] = metrics_easyocr['combined']
        if tesseract_available:
            scores['Tess'] = metrics_tesseract['combined']
        winner = max(scores, key=scores.get)
        
        # Print
        print(f"[{i+1:3d}/300] 🏆 Winner: {winner}")
        print(f"   GT:        {ground_truth}")
        print(f"   Yours:     {pred_yours[:50]}{'...' if len(pred_yours) > 50 else ''}")
        print(f"             CER:{metrics_yours['cer']:.3f} | Word:{metrics_yours['word_acc']:.3f} | Seq:{metrics_yours['seq_match']:.3f} | Score:{metrics_yours['combined']:.3f}")
        print(f"   ThaiTrOCR: {pred_otg[:50]}{'...' if len(pred_otg) > 50 else ''}")
        print(f"             CER:{metrics_otg['cer']:.3f} | Word:{metrics_otg['word_acc']:.3f} | Seq:{metrics_otg['seq_match']:.3f} | Score:{metrics_otg['combined']:.3f}")
        if easyocr_available:
            print(f"   EasyOCR:   {pred_easyocr[:50]}{'...' if len(pred_easyocr) > 50 else ''}")
            print(f"             CER:{metrics_easyocr['cer']:.3f} | Word:{metrics_easyocr['word_acc']:.3f} | Seq:{metrics_easyocr['seq_match']:.3f} | Score:{metrics_easyocr['combined']:.3f}")
        if tesseract_available:
            print(f"   Tesseract: {pred_tesseract[:50]}{'...' if len(pred_tesseract) > 50 else ''}")
            print(f"             CER:{metrics_tesseract['cer']:.3f} | Word:{metrics_tesseract['word_acc']:.3f} | Seq:{metrics_tesseract['seq_match']:.3f} | Score:{metrics_tesseract['combined']:.3f}")
        print()


# ==================== SUMMARY ====================
print("="*80)
print("📈 FINAL COMPARISON SUMMARY")
print("="*80)

model_names = {'yours': 'Your Model', 'openthaigpt': 'OpenThaiGPT', 'easyocr': 'EasyOCR', 'tesseract': 'Tesseract'}
active_models = ['yours', 'openthaigpt']
if easyocr_available:
    active_models.append('easyocr')
if tesseract_available:
    active_models.append('tesseract')

# Calculate averages
stats = {}
for name in active_models:
    res = [r for r in results[name] if r.get('combined', -1) >= 0]
    stats[name] = {
        'avg_cer': sum(r['cer'] for r in res) / len(res),
        'avg_char_acc': sum(r['char_acc'] for r in res) / len(res),
        'avg_word_acc': sum(r['word_acc'] for r in res) / len(res),
        'avg_seq_match': sum(r['seq_match'] for r in res) / len(res),
        'avg_combined': sum(r['combined'] for r in res) / len(res),
    }

# Count wins by combined score
win_counts = {name: 0 for name in active_models}
for i in range(len(sample_indices)):
    scores = {name: results[name][i].get('combined', -1) for name in active_models}
    scores = {k: v for k, v in scores.items() if v >= 0}
    if scores:
        winner = max(scores, key=scores.get)
        win_counts[winner] += 1

# Print detailed table
print(f"\n{'Model':<15} {'CER↓':>8} {'CharAcc↑':>10} {'WordAcc↑':>10} {'SeqMatch↑':>10} {'Combined↑':>10} {'Wins':>8}")
print("-"*75)
for name in active_models:
    s = stats[name]
    print(f"{model_names[name]:<15} {s['avg_cer']:>8.4f} {s['avg_char_acc']:>10.4f} {s['avg_word_acc']:>10.4f} {s['avg_seq_match']:>10.4f} {s['avg_combined']:>10.4f} {win_counts[name]:>8}")

# Overall winner
best_model = max(active_models, key=lambda x: stats[x]['avg_combined'])
print(f"\n🏆 OVERALL WINNER: {model_names[best_model]} (Combined Score: {stats[best_model]['avg_combined']:.4f})")

print("\n📊 Metrics Explanation:")
print("   • CER (↓ lower better): Character Error Rate - edit distance / GT length")
print("   • CharAcc (↑ higher better): % characters that appear in GT (position-independent)")
print("   • WordAcc (↑ higher better): % words that match GT")
print("   • SeqMatch (↑ higher better): Longest common subsequence ratio")
print("   • Combined (↑ higher better): Weighted score = 0.2×(1-CER) + 0.2×CharAcc + 0.25×WordAcc + 0.15×Ngram3 + 0.2×SeqMatch")
print("   • Wins: จำนวนครั้งที่ model ได้ Combined Score สูงสุดในแต่ละ sample")

print("\n✅ Comparison completed!")


🏆 4-WAY MODEL COMPARISON (Multi-Metric)
   Your Model vs OpenThaiGPT vs EasyOCR vs Tesseract

📦 Loading models...
   Loading OpenThaiGPT Thai-TrOCR...
   ✅ OpenThaiGPT loaded
   Loading EasyOCR...
   ✅ EasyOCR loaded
   Loading Tesseract...
   ✅ Tesseract loaded

📦 Loading test dataset...
✅ Loaded 10000 samples
🎲 Randomly selected 100 samples

📊 PREDICTIONS COMPARISON

[  1/300] 🏆 Winner: Easy
   GT:        ถ้าขาดความเข้าใจหรือขาดความระมัดระวังแล้วหน้าดินก็จะถูกน้ำพัดพาไปจนเหลือแต่ดินดาน
   Yours:     ถ้าขาดความเข้าใจหรือขาดนำสำเร็จแล้วจะถูกจะถูกพัดไป...
             CER:0.494 | Word:0.000 | Seq:0.531 | Score:0.404
   ThaiTrOCR: พันต้นน้ํานนเธียเรียลพัฒนาปมละป
             CER:0.877 | Word:0.000 | Seq:0.148 | Score:0.111
   EasyOCR:   กาบาดความเข้ำคหรือ ขาดภาามระมัดระจัง แล็ำหน้าดินกิ...
             CER:0.346 | Word:0.000 | Seq:0.691 | Score:0.483
   Tesseract: ล  7   สนจษี ส   ¢ | : ana Fae ANN มัดระจัง เพณ์ติ...
             CER:0.864 | Word:0.000 | Seq:0.259 | Score:0.164

[  2/30

In [47]:
# ============================================================
# Cell: Compare Models using JiWER Library
# ============================================================
# pip install jiwer

from datasets import load_dataset
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import random
import torch
import numpy as np
from PIL import Image
import io

# Install jiwer if needed
try:
    from jiwer import wer, cer, mer, wil, wip
except ImportError:
    import subprocess
    subprocess.check_call(['pip', 'install', 'jiwer'])
    from jiwer import wer, cer, mer, wil, wip

# Fix SSL
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

print("\n" + "="*80)
print("🏆 MODEL COMPARISON using JiWER Metrics")
print("   Your Model vs OpenThaiGPT vs EasyOCR vs Tesseract")
print("="*80)

# ==================== LOAD MODELS ====================
print("\n📦 Loading models...")

# 1. OpenThaiGPT
print("   Loading OpenThaiGPT Thai-TrOCR...")
openthaigpt_processor = TrOCRProcessor.from_pretrained('openthaigpt/thai-trocr')
openthaigpt_model = VisionEncoderDecoderModel.from_pretrained('openthaigpt/thai-trocr')
openthaigpt_model = openthaigpt_model.to(DEVICE)
openthaigpt_model.eval()
print("   ✅ OpenThaiGPT loaded")

# 2. EasyOCR
print("   Loading EasyOCR...")
try:
    import easyocr
    easyocr_reader = easyocr.Reader(['th', 'en'], gpu=torch.cuda.is_available())
    easyocr_available = True
    print("   ✅ EasyOCR loaded")
except Exception as e:
    print(f"   ⚠️ EasyOCR failed: {e}")
    easyocr_available = False

# 3. Tesseract
print("   Loading Tesseract...")
try:
    import pytesseract
    pytesseract.get_tesseract_version()
    tesseract_available = True
    print("   ✅ Tesseract loaded")
except Exception as e:
    print(f"   ⚠️ Tesseract failed: {e}")
    tesseract_available = False

# ==================== LOAD TEST DATASET ====================
print("\n📦 Loading test dataset...")
bypkt_ds = load_dataset("bypkt/thai_handwritten_datasets")["train"]
print(f"✅ Loaded {len(bypkt_ds)} samples")

random.seed(42)
sample_indices = random.sample(range(len(bypkt_ds)), 300)
print(f"🎲 Randomly selected 100 samples")

bypkt_dataset = ThaiHandwritingDatasetFixed(bypkt_ds, thai_tokenizer_fixed, image_processor)

# ==================== RUN COMPARISON ====================
print("\n" + "="*80)
print("📊 PREDICTIONS COMPARISON")
print("="*80 + "\n")

results = {'yours': [], 'openthaigpt': [], 'easyocr': [], 'tesseract': []}

with torch.no_grad():
    for i, idx in enumerate(sample_indices):
        raw_item = bypkt_ds[idx]
        ground_truth = raw_item['text']
        
        image = raw_item['image']
        if not isinstance(image, Image.Image):
            image = Image.open(io.BytesIO(image['bytes']))
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # 1. YOUR MODEL
        sample = bypkt_dataset[idx]
        pixel_values_yours = sample['pixel_values'].unsqueeze(0).to(DEVICE)
        generated_ids_yours = model.generate(
            pixel_values_yours, max_length=150, num_beams=4, early_stopping=True,
            no_repeat_ngram_size=3, repetition_penalty=1.5,
            pad_token_id=thai_tokenizer_fixed.pad_token_id,
            bos_token_id=thai_tokenizer_fixed.bos_token_id,
            eos_token_id=thai_tokenizer_fixed.eos_token_id,
        )
        pred_yours = thai_tokenizer_fixed.decode(generated_ids_yours[0], skip_special_tokens=True)
        
        # 2. OPENTHAIGPT
        pixel_values_otg = openthaigpt_processor(images=image, return_tensors="pt").pixel_values.to(DEVICE)
        generated_ids_otg = openthaigpt_model.generate(pixel_values_otg)
        pred_otg = openthaigpt_processor.batch_decode(generated_ids_otg, skip_special_tokens=True)[0]
        
        # 3. EASYOCR
        if easyocr_available:
            try:
                easyocr_result = easyocr_reader.readtext(np.array(image), detail=0)
                pred_easyocr = ''.join(easyocr_result) if easyocr_result else ''
            except:
                pred_easyocr = ''
        else:
            pred_easyocr = ''
        
        # 4. TESSERACT
        if tesseract_available:
            try:
                pred_tesseract = pytesseract.image_to_string(image, lang='tha+eng').strip()
                pred_tesseract = pred_tesseract.replace('\n', ' ').replace('  ', ' ')
            except:
                pred_tesseract = ''
        else:
            pred_tesseract = ''
        
        # Calculate JiWER metrics (handle empty strings)
        def safe_jiwer(ref, hyp):
            """Calculate JiWER metrics safely"""
            if not ref or not hyp:
                return {'wer': 1.0, 'cer': 1.0, 'mer': 1.0, 'wil': 1.0, 'wip': 0.0}
            try:
                return {
                    'wer': wer(ref, hyp),
                    'cer': cer(ref, hyp),
                    'mer': mer(ref, hyp),
                    'wil': wil(ref, hyp),
                    'wip': wip(ref, hyp),
                }
            except:
                return {'wer': 1.0, 'cer': 1.0, 'mer': 1.0, 'wil': 1.0, 'wip': 0.0}
        
        metrics_yours = safe_jiwer(ground_truth, pred_yours)
        metrics_otg = safe_jiwer(ground_truth, pred_otg)
        metrics_easyocr = safe_jiwer(ground_truth, pred_easyocr) if easyocr_available else None
        metrics_tesseract = safe_jiwer(ground_truth, pred_tesseract) if tesseract_available else None
        
        # Store results
        results['yours'].append({'gt': ground_truth, 'pred': pred_yours, **metrics_yours})
        results['openthaigpt'].append({'gt': ground_truth, 'pred': pred_otg, **metrics_otg})
        if easyocr_available:
            results['easyocr'].append({'gt': ground_truth, 'pred': pred_easyocr, **metrics_easyocr})
        if tesseract_available:
            results['tesseract'].append({'gt': ground_truth, 'pred': pred_tesseract, **metrics_tesseract})
        
        # Find winner by WIP (Word Information Preserved) - higher is better
        scores = {'Yours': metrics_yours['wip'], 'OTG': metrics_otg['wip']}
        if easyocr_available and metrics_easyocr:
            scores['Easy'] = metrics_easyocr['wip']
        if tesseract_available and metrics_tesseract:
            scores['Tess'] = metrics_tesseract['wip']
        winner = max(scores, key=scores.get)
        
        # Print
        print(f"[{i+1:3d}/300] 🏆 Winner: {winner}")
        print(f"   GT:        {ground_truth}")
        print(f"   Yours:     {pred_yours[:50]}{'...' if len(pred_yours) > 50 else ''}")
        print(f"             WER:{metrics_yours['wer']:.3f} | CER:{metrics_yours['cer']:.3f} | WIP:{metrics_yours['wip']:.3f}")
        print(f"   ThaiTrOCR: {pred_otg[:50]}{'...' if len(pred_otg) > 50 else ''}")
        print(f"             WER:{metrics_otg['wer']:.3f} | CER:{metrics_otg['cer']:.3f} | WIP:{metrics_otg['wip']:.3f}")
        if easyocr_available and metrics_easyocr:
            print(f"   EasyOCR:   {pred_easyocr[:50]}{'...' if len(pred_easyocr) > 50 else ''}")
            print(f"             WER:{metrics_easyocr['wer']:.3f} | CER:{metrics_easyocr['cer']:.3f} | WIP:{metrics_easyocr['wip']:.3f}")
        if tesseract_available and metrics_tesseract:
            print(f"   Tesseract: {pred_tesseract[:50]}{'...' if len(pred_tesseract) > 50 else ''}")
            print(f"             WER:{metrics_tesseract['wer']:.3f} | CER:{metrics_tesseract['cer']:.3f} | WIP:{metrics_tesseract['wip']:.3f}")
        print()

# ==================== SUMMARY ====================
print("="*80)
print("📈 FINAL COMPARISON SUMMARY (JiWER Metrics)")
print("="*80)

model_names = {'yours': 'Your Model', 'openthaigpt': 'OpenThaiGPT', 'easyocr': 'EasyOCR', 'tesseract': 'Tesseract'}
active_models = ['yours', 'openthaigpt']
if easyocr_available:
    active_models.append('easyocr')
if tesseract_available:
    active_models.append('tesseract')

# Calculate averages
stats = {}
for name in active_models:
    res = results[name]
    stats[name] = {
        'avg_wer': sum(r['wer'] for r in res) / len(res),
        'avg_cer': sum(r['cer'] for r in res) / len(res),
        'avg_mer': sum(r['mer'] for r in res) / len(res),
        'avg_wil': sum(r['wil'] for r in res) / len(res),
        'avg_wip': sum(r['wip'] for r in res) / len(res),
    }

# Count wins by WIP (higher is better)
win_counts = {name: 0 for name in active_models}
for i in range(len(sample_indices)):
    scores = {}
    for name in active_models:
        if i < len(results[name]):
            scores[name] = results[name][i]['wip']
    if scores:
        winner = max(scores, key=scores.get)
        win_counts[winner] += 1

# Print detailed table
print(f"\n{'Model':<15} {'WER↓':>8} {'CER↓':>8} {'MER↓':>8} {'WIL↓':>8} {'WIP↑':>8} {'Wins':>8}")
print("-"*70)
for name in active_models:
    s = stats[name]
    print(f"{model_names[name]:<15} {s['avg_wer']:>8.4f} {s['avg_cer']:>8.4f} {s['avg_mer']:>8.4f} {s['avg_wil']:>8.4f} {s['avg_wip']:>8.4f} {win_counts[name]:>8}")

# Overall winner by WIP
best_model = max(active_models, key=lambda x: stats[x]['avg_wip'])
print(f"\n🏆 OVERALL WINNER: {model_names[best_model]} (WIP: {stats[best_model]['avg_wip']:.4f})")

print("\n📊 JiWER Metrics Explanation:")
print("   • WER (↓ lower better): Word Error Rate - word-level edit distance")
print("   • CER (↓ lower better): Character Error Rate - char-level edit distance")
print("   • MER (↓ lower better): Match Error Rate - % of words that don't match")
print("   • WIL (↓ lower better): Word Information Lost - how much info is lost")
print("   • WIP (↑ higher better): Word Information Preserved - how much info is kept")
print("   • Wins: จำนวนครั้งที่ model ได้ WIP สูงสุดในแต่ละ sample")

print("\n✅ Comparison completed!")


🏆 MODEL COMPARISON using JiWER Metrics
   Your Model vs OpenThaiGPT vs EasyOCR vs Tesseract

📦 Loading models...
   Loading OpenThaiGPT Thai-TrOCR...
   ✅ OpenThaiGPT loaded
   Loading EasyOCR...
   ✅ EasyOCR loaded
   Loading Tesseract...
   ✅ Tesseract loaded

📦 Loading test dataset...
✅ Loaded 10000 samples
🎲 Randomly selected 100 samples

📊 PREDICTIONS COMPARISON

[  1/300] 🏆 Winner: Yours
   GT:        ถ้าขาดความเข้าใจหรือขาดความระมัดระวังแล้วหน้าดินก็จะถูกน้ำพัดพาไปจนเหลือแต่ดินดาน
   Yours:     ถ้าขาดความเข้าใจหรือขาดนำสำเร็จแล้วจะถูกจะถูกพัดไป...
             WER:1.000 | CER:0.494 | WIP:0.000
   ThaiTrOCR: พันต้นน้ํานนเธียเรียลพัฒนาปมละป
             WER:1.000 | CER:0.877 | WIP:0.000
   EasyOCR:   กาบาดความเข้ำคหรือ ขาดภาามระมัดระจัง แล็ำหน้าดินกิ...
             WER:3.000 | CER:0.346 | WIP:0.000
   Tesseract: ล  7   สนจษี ส   ¢ | : ana Fae ANN มัดระจัง เพณ์ติ...
             WER:14.000 | CER:0.864 | WIP:0.000

[  2/300] 🏆 Winner: Yours
   GT:        ...และเห็นภาพของหญิงสาวผู้

In [48]:
# ============================================================
# Cell: Compare Models using JiWER CER
# ============================================================
from datasets import load_dataset
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import random
import torch
import numpy as np
from PIL import Image
import io

# Install jiwer if needed
try:
    from jiwer import cer
except ImportError:
    import subprocess
    subprocess.check_call(['pip', 'install', 'jiwer'])
    from jiwer import cer

# Fix SSL
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

print("\n" + "="*80)
print("🏆 MODEL COMPARISON using JiWER CER")
print("   Your Model vs OpenThaiGPT vs EasyOCR vs Tesseract")
print("="*80)

# ==================== LOAD MODELS ====================
print("\n📦 Loading models...")

# 1. OpenThaiGPT
print("   Loading OpenThaiGPT Thai-TrOCR...")
openthaigpt_processor = TrOCRProcessor.from_pretrained('openthaigpt/thai-trocr')
openthaigpt_model = VisionEncoderDecoderModel.from_pretrained('openthaigpt/thai-trocr')
openthaigpt_model = openthaigpt_model.to(DEVICE)
openthaigpt_model.eval()
print("   ✅ OpenThaiGPT loaded")

# 2. EasyOCR
print("   Loading EasyOCR...")
try:
    import easyocr
    easyocr_reader = easyocr.Reader(['th', 'en'], gpu=torch.cuda.is_available())
    easyocr_available = True
    print("   ✅ EasyOCR loaded")
except Exception as e:
    print(f"   ⚠️ EasyOCR failed: {e}")
    easyocr_available = False

# 3. Tesseract
print("   Loading Tesseract...")
try:
    import pytesseract
    pytesseract.get_tesseract_version()
    tesseract_available = True
    print("   ✅ Tesseract loaded")
except Exception as e:
    print(f"   ⚠️ Tesseract failed: {e}")
    tesseract_available = False

# ==================== LOAD TEST DATASET ====================
print("\n📦 Loading test dataset...")
bypkt_ds = load_dataset("bypkt/thai_handwritten_datasets")["train"]
print(f"✅ Loaded {len(bypkt_ds)} samples")

random.seed(42)
sample_indices = random.sample(range(len(bypkt_ds)), 300)
print(f"🎲 Randomly selected 100 samples")

bypkt_dataset = ThaiHandwritingDatasetFixed(bypkt_ds, thai_tokenizer_fixed, image_processor)

# ==================== HELPER ====================
def safe_cer(ref, hyp):
    """Calculate CER safely"""
    if not ref or not hyp:
        return 1.0
    try:
        return cer(ref, hyp)
    except:
        return 1.0

# ==================== RUN COMPARISON ====================
print("\n" + "="*80)
print("📊 PREDICTIONS COMPARISON")
print("="*80 + "\n")

results = {'yours': [], 'openthaigpt': [], 'easyocr': [], 'tesseract': []}

with torch.no_grad():
    for i, idx in enumerate(sample_indices):
        raw_item = bypkt_ds[idx]
        ground_truth = raw_item['text']
        
        image = raw_item['image']
        if not isinstance(image, Image.Image):
            image = Image.open(io.BytesIO(image['bytes']))
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # 1. YOUR MODEL
        sample = bypkt_dataset[idx]
        pixel_values_yours = sample['pixel_values'].unsqueeze(0).to(DEVICE)
        generated_ids_yours = model.generate(
            pixel_values_yours, max_length=150, num_beams=4, early_stopping=True,
            no_repeat_ngram_size=3, repetition_penalty=1.5,
            pad_token_id=thai_tokenizer_fixed.pad_token_id,
            bos_token_id=thai_tokenizer_fixed.bos_token_id,
            eos_token_id=thai_tokenizer_fixed.eos_token_id,
        )
        pred_yours = thai_tokenizer_fixed.decode(generated_ids_yours[0], skip_special_tokens=True)
        cer_yours = safe_cer(ground_truth, pred_yours)
        
        # 2. OPENTHAIGPT
        pixel_values_otg = openthaigpt_processor(images=image, return_tensors="pt").pixel_values.to(DEVICE)
        generated_ids_otg = openthaigpt_model.generate(pixel_values_otg)
        pred_otg = openthaigpt_processor.batch_decode(generated_ids_otg, skip_special_tokens=True)[0]
        cer_otg = safe_cer(ground_truth, pred_otg)
        
        # 3. EASYOCR
        if easyocr_available:
            try:
                easyocr_result = easyocr_reader.readtext(np.array(image), detail=0)
                pred_easyocr = ''.join(easyocr_result) if easyocr_result else ''
            except:
                pred_easyocr = ''
            cer_easyocr = safe_cer(ground_truth, pred_easyocr)
        else:
            pred_easyocr = '[N/A]'
            cer_easyocr = -1
        
        # 4. TESSERACT
        if tesseract_available:
            try:
                pred_tesseract = pytesseract.image_to_string(image, lang='tha+eng').strip()
                pred_tesseract = pred_tesseract.replace('\n', ' ').replace('  ', ' ')
            except:
                pred_tesseract = ''
            cer_tesseract = safe_cer(ground_truth, pred_tesseract)
        else:
            pred_tesseract = '[N/A]'
            cer_tesseract = -1
        
        # Store results
        results['yours'].append({'gt': ground_truth, 'pred': pred_yours, 'cer': cer_yours})
        results['openthaigpt'].append({'gt': ground_truth, 'pred': pred_otg, 'cer': cer_otg})
        results['easyocr'].append({'gt': ground_truth, 'pred': pred_easyocr, 'cer': cer_easyocr})
        results['tesseract'].append({'gt': ground_truth, 'pred': pred_tesseract, 'cer': cer_tesseract})
        
        # Find winner (lowest CER)
        cers = {'Yours': cer_yours, 'OTG': cer_otg}
        if easyocr_available:
            cers['Easy'] = cer_easyocr
        if tesseract_available:
            cers['Tess'] = cer_tesseract
        winner = min(cers, key=cers.get)
        
        # Print
        print(f"[{i+1:3d}/300] 🏆 Winner: {winner}")
        print(f"   GT:        {ground_truth}")
        print(f"   Yours:     {pred_yours[:60]}{'...' if len(pred_yours) > 60 else ''} (CER: {cer_yours:.4f})")
        print(f"   ThaiTrOCR: {pred_otg[:60]}{'...' if len(pred_otg) > 60 else ''} (CER: {cer_otg:.4f})")
        if easyocr_available:
            print(f"   EasyOCR:   {pred_easyocr[:60]}{'...' if len(pred_easyocr) > 60 else ''} (CER: {cer_easyocr:.4f})")
        if tesseract_available:
            print(f"   Tesseract: {pred_tesseract[:60]}{'...' if len(pred_tesseract) > 60 else ''} (CER: {cer_tesseract:.4f})")
        print()

# ==================== SUMMARY ====================
print("="*80)
print("📈 FINAL COMPARISON SUMMARY (JiWER CER)")
print("="*80)

model_names = {'yours': 'Your Model', 'openthaigpt': 'OpenThaiGPT', 'easyocr': 'EasyOCR', 'tesseract': 'Tesseract'}
active_models = ['yours', 'openthaigpt']
if easyocr_available:
    active_models.append('easyocr')
if tesseract_available:
    active_models.append('tesseract')

# Calculate averages
stats = {}
for name in active_models:
    res = [r for r in results[name] if r['cer'] >= 0]
    stats[name] = {
        'avg_cer': sum(r['cer'] for r in res) / len(res),
        'exact_match': sum(1 for r in res if r['cer'] == 0),
    }

# Count wins (lowest CER)
win_counts = {name: 0 for name in active_models}
for i in range(len(sample_indices)):
    cers = {name: results[name][i]['cer'] for name in active_models if results[name][i]['cer'] >= 0}
    if cers:
        winner = min(cers, key=cers.get)
        win_counts[winner] += 1

# Print table
print(f"\n{'Model':<15} {'Avg CER↓':>12} {'Exact Match':>15} {'Wins':>10}")
print("-"*55)
for name in active_models:
    s = stats[name]
    print(f"{model_names[name]:<15} {s['avg_cer']:>12.4f} {s['exact_match']:>15} {win_counts[name]:>10}")

# Overall winner (lowest CER)
best_model = min(active_models, key=lambda x: stats[x]['avg_cer'])
print(f"\n🏆 OVERALL WINNER: {model_names[best_model]} (Avg CER: {stats[best_model]['avg_cer']:.4f})")

print("\n📊 Metrics Explanation:")
print("   • CER (↓ lower better): Character Error Rate from JiWER library")
print("   • Exact Match: จำนวน samples ที่ทายถูก 100% (CER = 0)")
print("   • Wins: จำนวนครั้งที่ model ได้ CER ต่ำสุดในแต่ละ sample")

print("\n✅ Comparison completed!")


🏆 MODEL COMPARISON using JiWER CER
   Your Model vs OpenThaiGPT vs EasyOCR vs Tesseract

📦 Loading models...
   Loading OpenThaiGPT Thai-TrOCR...
   ✅ OpenThaiGPT loaded
   Loading EasyOCR...
   ✅ EasyOCR loaded
   Loading Tesseract...
   ✅ Tesseract loaded

📦 Loading test dataset...
✅ Loaded 10000 samples
🎲 Randomly selected 100 samples

📊 PREDICTIONS COMPARISON

[  1/300] 🏆 Winner: Easy
   GT:        ถ้าขาดความเข้าใจหรือขาดความระมัดระวังแล้วหน้าดินก็จะถูกน้ำพัดพาไปจนเหลือแต่ดินดาน
   Yours:     ถ้าขาดความเข้าใจหรือขาดนำสำเร็จแล้วจะถูกจะถูกพัดไปไปไปอย่างช... (CER: 0.4938)
   ThaiTrOCR: พันต้นน้ํานนเธียเรียลพัฒนาปมละป (CER: 0.8765)
   EasyOCR:   กาบาดความเข้ำคหรือ ขาดภาามระมัดระจัง แล็ำหน้าดินกิจะกานามัดพ... (CER: 0.3457)
   Tesseract: ล  7   สนจษี ส   ¢ | : ana Fae ANN มัดระจัง เพณ์ตินก จนเชถี๊... (CER: 0.8642)

[  2/300] 🏆 Winner: Yours
   GT:        ...และเห็นภาพของหญิงสาวผู้หนึ่งยืนนิ่งสงบอยู่ระหว่างช่องประตูหน้าบ้านที่เปิดกว้าง...
   Yours:     ...และภาพของหญิงสาวหญิงสาวผู้หนึ่งย